In [1]:
import pandas as pd
import backtrader as bt
from utils.gridsearch import gridsearch
from utils.read2df import read2df

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT']

freqs = {'1m': 1, '3m': 3, '5m': 5, '15m': 15, '30m': 30}

# Download Data from [binance-public-data](https://github.com/binance/binance-public-data/tree/master/python)

Download BTCUSDT and ETHUSDT for all available history for intervals of 1m, 3m, 5m, 15m, 30m


In [8]:
%%capture
!python binance-public-data/python/download-kline.py -t spot -s BTCUSDT ETHUSDT -i 1m 3m 5m 15m 30m -skip-daily 1
print("Downloading Completed")

SyntaxError: unterminated string literal (detected at line 2) (3720783001.py, line 2)

# Read Downloaded Data

The downloaded data can be found in `binance-public-data/data/`

In [3]:
dfs = read2df(symbols, freqs)
dfs[0]

Load dataframes into datafeeds based on frequencies

In [6]:
datafeeds_eth = []
datafeeds_btc = []

for idx, freq in enumerate(freqs):
    datafeeds_eth.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='ETHUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    datafeeds_btc.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='BTCUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    print(freq)

1m
3m
5m
15m
30m


# Define Trading Strategy

In [42]:
class PairTrading(bt.Strategy):
    params = dict(
        OPEN_THRE=5,
        CLOS_THRE=0.1,
        period=60
    )

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            if order.isbuy():
                print(f"Buy {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")
            else:
                print(f"Sell {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            print('%s ,' % order.Status[order.status])
            pass

    def __init__(self):
        self.data0 = self.datas[0]
        self.data1 = self.datas[1]

        # self.qty0 = self.qty1 = 0

        self.transform = bt.indicators.OLS_TransformationN(self.data1, self.data0, period=self.p.period)
        # self.ols = bt.indicators.OLS_Slope_InterceptN(self.data1, self.data0, period=self.p.period)

        # self.slope = self.ols.slope
        # self.intercept = self.ols.intercept

        # self.spread = self.transform.spread
        # self.spread_std = self.transform.spread_std
        # self.spread_mean = self.transform.spread_mean
        self.zscore = self.transform.zscore

        # -1 for short data1/data0, 1 for long data1/data0, 0 for no position
        self.position_status = 0

    def next(self):
        # print(f'Right now the zscore is {self.transform.zscore[0]}, and the position is {self.position_status}')

        ratio = self.data1.close[0] / self.data0.close[0]

        if self.zscore[0] > self.params.OPEN_THRE and self.position_status == 0:
            print("------")
            print("long data1 and short data0")
            self.position_status = -1

            self.sell(data=self.data0, size=ratio)
            self.buy(data=self.data1, size=1)
            
            # self.qty0 = -ratio
            # self.qty1 = 1
    
           elif self.zscore[0] < -self.params.OPEN_THRE and self.position_status == 0:
            print("------")
            print("long data0 and short data1")
            self.position_status = 1
            self.sell(data=self.data0, size=ratio)
            self.buy(data=self.data1, size=1)

            # self.qty0 = -ratio
            # self.qty1 = 1

        elif abs(self.zscore[0]) < self.params.CLOS_THRE and self.position_status != 0:
            print("------")
            print("close position")
            self.position_status = 0
            self.close(data=self.data0)
            self.close(data=self.data1)

            # self.qty0 = self.qty1 = 0

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')

[                  time      open      high       low     close      volume  \
 0        1502942459999   4261.48   4261.48   4261.48   4261.48    1.775183   
 1        1502942459999    301.13    301.13    301.13    301.13    0.426430   
 2        1502942519999   4261.48   4261.48   4261.48   4261.48    0.000000   
 3        1502942519999    301.13    301.13    301.13    301.13    2.757870   
 4        1502942579999   4280.56   4280.56   4280.56   4280.56    0.261074   
 ...                ...       ...       ...       ...       ...         ...   
 6292443  1693526279999   1646.26   1646.26   1645.87   1645.88   18.323700   
 6292444  1693526339999  25936.75  25941.60  25936.75  25937.85   15.876600   
 6292445  1693526339999   1645.88   1646.04   1645.76   1645.76   77.918700   
 6292446  1693526399999  25937.85  25940.78  25937.84  25940.78    7.793620   
 6292447  1693526399999   1645.76   1645.77   1645.76   1645.76  107.315300   
 
              tic itvl                datetime  
 